In [1]:
from collections import namedtuple
import pandas as pd, numpy as np
import dash
from dash import dcc, html
import plotly.express as px

/Users/mariaolivacalero/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('MyData/history.csv', index_col = 0)
df.shape

(30272, 26)

In [3]:
df.head()

,name,endTime,artistName,trackName,msPlayed,datetime,danceability,energy,key,loudness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,album_id,album_name
number,,,,,,,,,,,,,,,,,,,,,
0,Que Alegria Mas Tonta___Pereza,2022-11-09 00:01,Pereza,Que Alegria Mas Tonta,29858,2022-11-09 00:01:00,0.628,0.911,10,-3.177,...,128.861,audio_features,09pHfOLKeTx3txsUtxPpvR,spotify:track:09pHfOLKeTx3txsUtxPpvR,https://api.spotify.com/v1/tracks/09pHfOLKeTx3...,https://api.spotify.com/v1/audio-analysis/09pH...,181680,4,6TFvbSAqnlvjOSkuryxUKe,Animales
1,Bañarnos en Vaqueros___Sofia Ellar,2022-11-09 00:02,Sofia Ellar,Bañarnos en Vaqueros,221630,2022-11-09 00:02:00,0.645,0.614,6,-6.953,...,95.978,audio_features,6xvTbeuhoQMzRqmFeZr3tQ,spotify:track:6xvTbeuhoQMzRqmFeZr3tQ,https://api.spotify.com/v1/tracks/6xvTbeuhoQMz...,https://api.spotify.com/v1/audio-analysis/6xvT...,221631,4,2XM0HxGntoa0QJcSX9Gk9f,Bañarnos en Vaqueros
2,Para Que el Mundo lo Vea___Arnau Griso,2022-11-09 00:06,Arnau Griso,Para Que el Mundo lo Vea,232747,2022-11-09 00:06:00,0.663,0.737,8,-6.296,...,90.987,audio_features,4dqR44eVmK4S9gbJPcxQWW,spotify:track:4dqR44eVmK4S9gbJPcxQWW,https://api.spotify.com/v1/tracks/4dqR44eVmK4S...,https://api.spotify.com/v1/audio-analysis/4dqR...,232747,4,4DImcSLrz8KPotPPFctfiv,Revolución Bananera
3,No Puedo Vivir Sin Ti___El Canto Del Loco,2022-11-09 00:09,El Canto Del Loco,No Puedo Vivir Sin Ti,214960,2022-11-09 00:09:00,0.666,0.834,0,-3.314,...,129.982,audio_features,3K8BeABgLXZ3JHhdM3rZBx,spotify:track:3K8BeABgLXZ3JHhdM3rZBx,https://api.spotify.com/v1/tracks/3K8BeABgLXZ3...,https://api.spotify.com/v1/audio-analysis/3K8B...,214960,4,3ccUQA3LcKnQN28rHI597A,Por Mi y por Todos Mis Compañeros
4,Copenhague___Eva B,2022-11-09 00:11,Eva B,Copenhague,121584,2022-11-09 00:11:00,0.576,0.787,9,-6.363,...,93.001,audio_features,40IWFXz1cOoeuURmOeOILC,spotify:track:40IWFXz1cOoeuURmOeOILC,https://api.spotify.com/v1/tracks/40IWFXz1cOoe...,https://api.spotify.com/v1/audio-analysis/40IW...,151774,4,766C7r1vSvIalNiUcGE1fX,Copenhague


In [4]:
#tracks playing time
df['secPlayed'] = df['msPlayed'] / 1000
df = df[df.columns[:-1].insert(4, df.columns[-1])] #moving seconds column to proper place
df = df[df.secPlayed > 60] #removing songs that were played for less than 60 secs 
                            # me quito 60000 reproducciones
df.shape

(23617, 27)

In [5]:
#identifying the month
df['month'] = df.endTime.str.split('-').apply(lambda x: (x[0], x[1]))

In [6]:
#listing months and features
months = list(set(df.month.values))
months.sort()
features = ['danceability', 'energy', 'speechiness', 'instrumentalness', 'valence']

In [7]:
#standardizing features (we're interested not in their absolute value, but in how each changed over time)
for feature in features:
    df[f'{feature}_zscore'] = ( df[feature] - df[feature].mean() ) / df[feature].std()

In [8]:

#making sure we standardized correctly: mean is 0 and std is 1
df[[feature + '_zscore' for feature in features]].describe().loc['mean':'std'].T

,mean,std
danceability_zscore,-4.055602e-16,1.0
energy_zscore,-1.059030e-16,1.0
speechiness_zscore,1.083099e-16,1.0
instrumentalness_zscore,2.406886e-17,1.0
valence_zscore,-2.202300e-16,1.0


In [9]:
#features averages by month
Month = namedtuple('Month', features)
avg_features_months = []
for month in months:
    df_month = df[df['month'] == month]
    avg_features = df_month.describe().loc['mean'][[feature + '_zscore' for feature in features]]
    month = Month(*avg_features)
    avg_features_months.append(month)

In [10]:
#labelling months
month_labels = [f'{month[1]}/{month[0]}' for month in months]
month_labels_short = [m[:3]+m[-2:] for m in month_labels]

In [123]:
import plotly.graph_objects as go

# Sample data
features = ['valence', 'energy']
x = [-1] + [x for x in range(13)]

# Create traces for each feature
traces = []
for feature in features:
    y = [getattr(month, feature) for month in avg_features_months]
    traces.append(go.Scatter(x=x, y=y, mode='lines+markers', name=feature))

# Create the layout
layout = go.Layout(
    title='My mood in 2019 (According to Spotify)',
    xaxis=dict(
        title='Months',
        tickmode='array',
        tickvals=list(range(13)),
        ticktext=month_labels_short,
    ),
    yaxis=dict(
        title='Mood',
    ),
    showlegend=True,
)

# Add annotations
annotations = [
    dict(x=1, y=0.4, xref="x", yref="y", text="Studying", showarrow=True, arrowhead=5, ax=0, ay=-40, font=dict(size=14)),
    dict(x=4.15, y=0.4, xref="x", yref="y", text="Graduating", showarrow=True, arrowhead=5, ax=0, ay=-40, font=dict(size=14)),
    dict(x=6.5, y=0.4, xref="x", yref="y", text="Unemployed", showarrow=True, arrowhead=5, ax=0, ay=-40, font=dict(size=14)),
    dict(x=10.35, y=0.4, xref="x", yref="y", text="First Job", showarrow=True, arrowhead=5, ax=0, ay=-40, font=dict(size=14)),
]

layout['annotations'] = annotations

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()



In [27]:
# converting ms to minute and extracting date from datetime column
df['mins_played'] = df.apply(lambda x: round(x['msPlayed']/60000,2), axis=1)
df['date'] = df.apply(lambda x: pd.to_datetime(x['datetime'][:10],format='%Y-%m-%d'),axis=1)

# calculate the daily streaming time length 
daily_length = df.groupby('date',as_index=True).sum()

# create new date series for displaying time series data
idx = pd.DataFrame(pd.date_range(min(df.date), max(df.date)),columns=['date'])
idx['date'] = idx.apply(lambda x: pd.to_datetime(x['date'],format='%Y-%m-%d'),axis=1)

# use new date series to display the daily streaming time
new_daily_length = pd.merge(idx, daily_length, how='left', left_on='date', right_on = 'date', copy=False)

# getting rid of columns except for date and time
new_daily_length = new_daily_length.drop(new_daily_length.loc[:, 'msPlayed':'time_signature'], axis=1)




In [28]:
new_daily_length.head()


,date,name,endTime,artistName,trackName,secPlayed,album_id,album_name,month,danceability_zscore,energy_zscore,speechiness_zscore,instrumentalness_zscore,valence_zscore,mins_played
0,2022-11-09,Bañarnos en Vaqueros___Sofia EllarPara Que el ...,2022-11-09 00:022022-11-09 00:062022-11-09 00:...,Sofia EllarArnau GrisoEl Canto Del LocoEva BMa...,Bañarnos en VaquerosPara Que el Mundo lo VeaNo...,10799.802,2XM0HxGntoa0QJcSX9Gk9f4DImcSLrz8KPotPPFctfiv3c...,Bañarnos en VaquerosRevolución BananeraPor Mi ...,"(2022, 11, 2022, 11, 2022, 11, 2022, 11, 2022,...",-6.742601,-13.403686,-7.594766,14.615580,-0.815722,179.99
1,2022-11-10,"Quevedo: Bzrp Music Sessions, Vol. 52___Bizarr...",2022-11-10 21:362022-11-10 21:382022-11-10 21:...,BizarrapFacundo MajdalaniManuel TurizoTers,"Quevedo: Bzrp Music Sessions, Vol. 52PegaoLa B...",671.424,4PNqWiJAfjj32hVvlchV5u0ITiVEYDdB5CPppQyyUEcL1T...,"Quevedo: Bzrp Music Sessions, Vol. 52PegaoLa B...","(2022, 11, 2022, 11, 2022, 11, 2022, 11)",3.457092,1.653061,0.585892,-0.281187,1.992298,11.19
2,2022-11-11,I'm Yours___Jason MrazHigh On Life (feat. Bonn...,2022-11-11 11:382022-11-11 11:432022-11-11 11:...,Jason MrazMartin GarrixStephen DawesBruno Mars...,I'm YoursHigh On Life (feat. Bonn)Teenage Drea...,3895.919,04G0YylSjvDQZrjOfE5jA51GUfof1gHsqYjoHFym3aim1U...,We Sing. We Dance. We Steal Things.High On Lif...,"(2022, 11, 2022, 11, 2022, 11, 2022, 11, 2022,...",-13.372472,-28.882529,-11.407135,-2.884921,-10.583172,64.92
3,2022-11-12,Teenage Dream___Stephen DawesTeenage Dream___S...,2022-11-12 11:252022-11-12 11:282022-11-12 11:...,Stephen DawesStephen DawesStephen DawesStephen...,Teenage DreamTeenage DreamTeenage DreamTeenage...,5484.930,1UH6aVsmnWRjsB5Tq0qUhF1UH6aVsmnWRjsB5Tq0qUhF1U...,Teenage DreamTeenage DreamTeenage DreamTeenage...,"(2022, 11, 2022, 11, 2022, 11, 2022, 11, 2022,...",0.091120,-20.637692,-10.877145,-4.163233,-10.926665,91.39
4,2022-11-13,"Quevedo: Bzrp Music Sessions, Vol. 52___Bizarr...",2022-11-13 12:302022-11-13 12:312022-11-13 12:...,BizarrapReyanna MariaMalmö 040Malmö 040Malmö 0...,"Quevedo: Bzrp Music Sessions, Vol. 52So Pretty...",1044.892,4PNqWiJAfjj32hVvlchV5u3K1FmvfuKFjry1x6sL405k4k...,"Quevedo: Bzrp Music Sessions, Vol. 52So Pretty...","(2022, 11, 2022, 11, 2022, 11, 2022, 11, 2022,...",1.560325,-1.458658,0.593503,-0.341605,-3.213744,17.44


In [16]:
import datetime
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np
from dash import dcc
from dash import html
import dash

	

In [115]:
def display_year(
    z,
    year: int = None,
    month_lines: bool = True,
    fig=None,
    row: int = None
):
    
    if year is None:
        year = datetime.datetime.now().year
        
    d1 = datetime.date(year, 1, 1)
    d2 = datetime.date(year, 12, 31)

    number_of_days = (d2-d1).days + 1
    
    data = np.ones(number_of_days) * np.nan
    data[:len(z)] = z
    

    d1 = datetime.date(year, 1, 1)
    d2 = datetime.date(year, 12, 31)

    delta = d2 - d1
    
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    month_days =   [31,    28,    31,     30,    31,     30,    31,    31,    30,    31,    30,    31]
    if number_of_days == 366:  # leap year
        month_days[1] = 29
    month_positions = (np.cumsum(month_days) - 15)/7

    dates_in_year = [d1 + datetime.timedelta(i) for i in range(delta.days+1)] # list with datetimes for each day a year
    weekdays_in_year = [i.weekday() for i in dates_in_year] # gives [0,1,2,3,4,5,6,0,1,2,3,4,5,6,…] (ticktext in xaxis dict translates this to weekdays
    
    weeknumber_of_dates = []
    for i in dates_in_year:
        inferred_week_no = int(i.strftime("%V"))
        if inferred_week_no >= 52 and i.month == 1:
            weeknumber_of_dates.append(0)
        elif inferred_week_no == 1 and i.month == 12:
            weeknumber_of_dates.append(53)
        else:
            weeknumber_of_dates.append(inferred_week_no)
    
    text = [str(i) for i in dates_in_year] #gives something like list of strings like ‘2018-01-25’ for each date. Used in data trace to make good hovertext.
    #4cc417 green #347c17 dark green
    colorscale=[[False, '#eeeeee'], [True, '#76cf63']]
    
    # handle end of year
    

    data = [
        go.Heatmap(
            x=weeknumber_of_dates,
            y=weekdays_in_year,
            z=data,
            text=text,
            hoverinfo='text',
            xgap=3, # this
            ygap=3, # and this is used to make the grid-like apperance
            showscale=False,
            colorscale=colorscale
        )
    ]
    
        
    if month_lines:
        kwargs = dict(
            mode='lines',
            line=dict(
                color='#9e9e9e',
                width=1,
            ),
            hoverinfo='skip',
        )
        
        for date, dow, wkn in zip(
            dates_in_year, weekdays_in_year, weeknumber_of_dates
        ):
            if date.day == 1:
                data += [
                    go.Scatter(
                        x=[wkn-.5, wkn-.5],
                        y=[dow-.5, 6.5],
                        **kwargs,
                    )
                ]
                if dow:
                    data += [
                    go.Scatter(
                        x=[wkn-.5, wkn+.5],
                        y=[dow-.5, dow - .5],
                        **kwargs,
                    ),
                    go.Scatter(
                        x=[wkn+.5, wkn+.5],
                        y=[dow-.5, -.5],
                        **kwargs,
                    )
                ]
                    
                    
    layout = go.Layout(
        title='My Spotify Activity',
        height=250,
        yaxis=dict(
            showline=False, showgrid=False, zeroline=False,
            tickmode='array',
            ticktext=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
            tickvals=[0, 1, 2, 3, 4, 5, 6],
            autorange="reversed",
        ),
        xaxis=dict(
            showline=False, showgrid=False, zeroline=False,
            tickmode='array',
            ticktext=month_names,
            tickvals=month_positions,
        ),
        font={'size':10, 'color':'#9e9e9e'},
        plot_bgcolor=('#fff'),
        margin = dict(t=40),
        showlegend=False,
    )

    if fig is None:
        fig = go.Figure(data=data, layout=layout)
    else:
        fig.add_traces(data, rows=[(row+1)]*len(data), cols=[1]*len(data))
        fig.update_layout(layout)
        fig.update_xaxes(layout['xaxis'])
        fig.update_yaxes(layout['yaxis'])

    
    return fig


def display_years(z, years):
    
    day_counter = 0
    
    fig = make_subplots(rows=len(years), cols=1, subplot_titles=years)
    for i, year in enumerate(years):
        d1 = datetime.date(year, 1, 1)
        d2 = datetime.date(year, 12, 31)
        
        number_of_days = (d2-d1).days + 1
        data = z[day_counter : day_counter + number_of_days]
        
        display_year(data, year=year, fig=fig, row=i)
        fig.update_layout(height=250*len(years))
        day_counter += number_of_days
    return fig



In [116]:
z = [0]* 312
for i in new_daily_length["secPlayed"]:
    z.append(i)
len(z)
display_years(z,(2022,2023))


In [95]:

z.append(new_daily_length["secPlayed"])

313

In [125]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

# Assuming 'avg_features_months' and 'month_labels_short' are defined

# Features to display
features = ['danceability', 'energy','valence','speechiness','instrumentalness']


# Create traces for each feature
traces = []
for feature in features:
    y = [getattr(month, feature) for month in avg_features_months]
    traces.append(go.Scatter(x=x, y=y, mode='lines+markers', name=feature))

# Create the layout
layout = go.Layout(
    title='My mood during last year (According to Spotify)',
    xaxis=dict(
        title='Months',
        tickmode='array',
        tickvals=list(range(13)),
        ticktext=month_labels_short,
    ),
    yaxis=dict(
        title='Mood',
    ),
    showlegend=True,
)

# Add annotations
annotations = [
    dict(x=1, y=0.4, xref="x", yref="y", text="Studying", showarrow=True, arrowhead=5, ax=0, ay=-40, font=dict(size=14)),
    dict(x=4.15, y=0.4, xref="x", yref="y", text="Graduating", showarrow=True, arrowhead=5, ax=0, ay=-40, font=dict(size=14)),
    dict(x=6.5, y=0.4, xref="x", yref="y", text="Unemployed", showarrow=True, arrowhead=5, ax=0, ay=-40, font=dict(size=14)),
    dict(x=10.35, y=0.4, xref="x", yref="y", text="First Job", showarrow=True, arrowhead=5, ax=0, ay=-40, font=dict(size=14)),
]

layout['annotations'] = annotations

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Create the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div(children=[
    # Dropdown selector for features
    dcc.Dropdown(
        id='feature-selector',
        options=[{'label': feature, 'value': feature} for feature in features],
        value=features,  # Initial selected features
        multi=True  # Allow multiple selection
    ),
    # Graph component
    dcc.Graph(
        id='mood-graph',
        figure=fig
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [120]:
Month

__main__.Month